In [1]:
import os, sys
os.environ['HADOOP_CONF_DIR'] = '/etc/hadoop/conf'
os.environ['YARN_CONF_DIR'] = '/etc/hadoop/conf'
os.environ['PYSPARK_PYTHON'] =  'python3.9'
os.environ['PYSPARK_DRIVER_PYTHON'] = 'python3.9'
os.environ['HADOOP_USER_NAME']='ssenigov'

from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf

In [2]:
conf = (SparkConf().setAppName('RepartitionWithColHash').setMaster('yarn') \
    .set('spark.sql.adaptive.enabled', 'false'))

spark = SparkSession.builder.config(conf=conf).getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/24 20:01:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/12/24 20:01:23 WARN DomainSocketFactory: The short-circuit local reads feature cannot be used because libhadoop cannot be loaded.
24/12/24 20:01:24 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [3]:
sc = spark.sparkContext
print("app_id".ljust(40), sc.applicationId)

app_id                                   application_1727681258360_0051


In [4]:
from pyspark.sql.functions import spark_partition_id, count, collect_list, col
# create DataFrame of length 40  
data = [ [k] for k in range(0, 40) ] # transform to 2-D list
df = spark.sparkContext.parallelize(data).toDF(["item"])
# create a surrogate key to partition by - reminder of division by 5 (modulo)
df = df.withColumn('partition_key', col('item') % 5) 

In [5]:
# repartition to 4 partitions and by column 'partition_key'
df4 = df.repartition(4, col('partition_key')).withColumn("partition_num", spark_partition_id())

df4.groupBy(["partition_num", "partition_key"]).agg(
      collect_list('item').alias('items_in_partition')\
   ).orderBy("partition_num").show(truncate=False)

+-------------+-------------+------------------------------+
|partition_num|partition_key|items_in_partition            |
+-------------+-------------+------------------------------+
|0            |2            |[2, 7, 12, 17, 22, 27, 32, 37]|
|0            |4            |[4, 9, 14, 19, 24, 29, 34, 39]|
|1            |0            |[20, 25, 30, 35, 0, 5, 10, 15]|
|1            |1            |[21, 26, 31, 36, 1, 6, 11, 16]|
|3            |3            |[23, 28, 33, 38, 3, 8, 13, 18]|
+-------------+-------------+------------------------------+



In [6]:
# repartition to 6 partitions and by column 'partition_key'
df6 = df.repartition(6, col('partition_key')).withColumn("partition_num", spark_partition_id())

df6.groupBy(["partition_num", "partition_key"]).agg(
      collect_list('item').alias('items_in_partition')\
   ).orderBy("partition_num").show(truncate=False)

+-------------+-------------+------------------------------+
|partition_num|partition_key|items_in_partition            |
+-------------+-------------+------------------------------+
|1            |0            |[20, 25, 30, 35, 0, 5, 10, 15]|
|2            |2            |[2, 7, 12, 17, 22, 27, 32, 37]|
|2            |4            |[4, 9, 14, 19, 24, 29, 34, 39]|
|3            |3            |[3, 8, 13, 18, 23, 28, 33, 38]|
|5            |1            |[21, 26, 31, 36, 1, 6, 11, 16]|
+-------------+-------------+------------------------------+



In [6]:
spark.stop()